In [2]:
import sys
import glob
import pytorch_lightning as pl
import torch
import pickle
from datasets import Dataset, DatasetDict, load_from_disk
from functools import partial
from transformers.models.llama.tokenization_llama import LlamaTokenizer
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import sentencepiece as spm
import os
from multiprocessing import Pool
from tqdm import tqdm
from datetime import datetime
from pytorch_lightning import seed_everything
import random
import gc
import h5py
import numpy as np

seed_everything(42)

/opt/rh/rh-python38/root/usr/local/lib64/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
2023-10-15 17:51:48.266155: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Global seed set to 42


42

In [46]:
text1 = ['GLTNAFIASAPAREVRYDGVITPANANYRFMGGDKGGSLTVGSHLTGSNMVTIGPMGVVVFTNNNDYTGNTFIMGGGTLQLGSNTAWGSLPN\n',
        'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFPLSPAQLGIWYAQHLDPQVPITIAQYVDLHGALDVEVLERASIDASHELGSGFLRIVERDGEPLQYV\n',
        'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFP\n',
        'MDRLDFGGGE\n']
text2 = ['GLTNAFIASAPAREVRYDGVITPANANYRFMGGDKGGSLTVGSHLTGSNMVTIGPMGVVVFTNNNDYTGNTFIMGGGTLQLGSNTAWGSLPN\n',
        'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFPLSPAQLGIWYAQHLDPQVPITIAQYVDLHGALDVEVLERASIDASHELGSGFLRIVERDGEPLQYV\n',
        'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFP\n',
        'MDRLDFGGGE\n']

In [83]:
def standalone_tokenize_function(batch_text1, batch_text2, max_sequence_length):
    global global_tokenizer

    batch_output = []

    # Assuming batch_text1 and batch_text2 are lists of sequences
    for s1, s2 in zip(batch_text1, batch_text2):
        protein_1 = global_tokenizer.encode(s1)
        protein_2 = global_tokenizer.encode(s2)

        # Here, we're not looping over the tokens but instead directly working with tokenized sequences.
        if len(protein_1) + len(protein_2) + 2 <= max_sequence_length:
            batch_output.append([global_tokenizer.bos_id()] + protein_1 + [global_tokenizer.eos_id()] + protein_2)
            batch_output.append([global_tokenizer.bos_id()] + protein_2 + [global_tokenizer.eos_id()] + protein_1)

    return batch_output
a = standalone_tokenize_function(text1, text2, 10)
print(a)

[[1, 854, 1642, 653, 91, 2, 854, 1642, 653, 91], [1, 854, 1642, 653, 91, 2, 854, 1642, 653, 91]]


In [67]:
def standalone_tokenize_function(s1, s2, max_sequence_length):
    global global_tokenizer
    try:
        protein_1 = global_tokenizer.encode(s1)
        protein_2 = global_tokenizer.encode(s2)
        batch_output = []
        for protein1, protein2 in zip(protein_1, protein_2):
            if len(protein1) + len(protein2) + 2 > max_sequence_length:
                pass
            else:
                batch_output.append(
                    [global_tokenizer.bos_id()] + protein1 + [global_tokenizer.eos_id()] + protein2)
                batch_output.append(
                    [global_tokenizer.bos_id()] + protein2 + [global_tokenizer.eos_id()] + protein1)
        return batch_output
    except Exception as e:
        raise ValueError(f"Error during tokenization of string fsl: fel")

a = standalone_tokenize_function(text1, text2, 10)
print(a)

[[1, 854, 1642, 653, 91, 2, 854, 1642, 653, 91], [1, 854, 1642, 653, 91, 2, 854, 1642, 653, 91]]


In [113]:
class SingleSequenceDataset(Dataset):
    def __init__(self, sequence_strs_1, sequence_strs_2, tokenizer, max_sequence_length):
        self.sequence_strs_1 = sequence_strs_1
        self.sequence_strs_2 = sequence_strs_2
        self.tokenizer = tokenizer
        global global_tokenizer
        global_tokenizer = tokenizer
        self.max_sequence_length = max_sequence_length
        self.tokenized_sequences = standalone_tokenize_function(sequence_strs_1, sequence_strs_2, self.max_sequence_length)

    def __len__(self):
        """
        returns the number of items in the dataset (number of pairs)
        """
        return 2* len(self.sequence_strs_1)

    def __getitem__(self, idx):
        print(self.sequence_strs_1, idx)
        s1 = self.sequence_strs_1[idx]
        s2 = self.sequence_strs_2[idx]
        tokenized_pairs = standalone_tokenize_function(s1, s2, self.max_sequence_length)
        return tokenized_pairs

def dynamic_batching_and_padding_collate_fn(batch, max_tokens_per_batch=512):
    batches_grouped_by_token_limit = []
    current_batch = []
    current_token_count = 0

    for tokenized_sequence in sorted(batch, key=len, reverse=True):  # Sorting for efficient batching
        if current_token_count + len(tokenized_sequence) > max_tokens_per_batch:
            batches_grouped_by_token_limit.append(current_batch)
            current_batch = []
            current_token_count = 0

        current_batch.append(tokenized_sequence)
        current_token_count += len(tokenized_sequence)

    if current_batch:
        batches_grouped_by_token_limit.append(current_batch)

    padded_batches = []
    for sequences in batches_grouped_by_token_limit:
        # Padding each batch to the length of its longest sequence.
        max_length = max(len(seq) for seq in sequences)
        padded_sequences = [seq + [global_tokenizer.pad_token_id()] * (max_length - len(seq)) for seq in sequences]
        padded_batches.append(torch.tensor(padded_sequences))

    return padded_batches  # List of tensors, where each tensor is a padded batch


In [114]:
text1 = ['GLTNAFIASAPAREVRYDGVITPANANYRFMGGDKGGSLTVGSHLTGSNMVTIGPMGVVVFTNNNDYTGNTFIMGGGTLQLGSNTAWGSLPN\n',
        'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFPLSPAQLGIWYAQHLDPQVPITIAQYVDLHGALDVEVLERASIDASHELGSGFLRIVERDGEPLQYV\n',
        'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFP\n',
        'MDRLDFGGGE\n',
        'APVPVSGQPVSSE\n']
text2 = ['GLTNAFIASAPAREVRYDGVITPANANYRFMGGDKGGSLTVGSHLTGSNMVTIGPMGVVVFTNNNDYTGNTFIMGGGTLQLGSNTAWGSLPN\n',
        'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFPLSPAQLGIWYAQHLDPQVPITIAQYVDLHGALDVEVLERASIDASHELGSGFLRIVERDGEPLQYV\n',
        'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFP\n',
        'MDRLDFGGGE\n',
        'APVPVSGQPVSSE\n']
tokenizer_path = '/data/rozen/home/e0833634/lama/protllama/batch_script/'
tokenizer = spm.SentencePieceProcessor(model_file=tokenizer_path+"protein_8k.model")

In [115]:
batch_size = 2  # This is the number of sequences.
train_dataloader = DataLoader(
    dataset=SingleSequenceDataset(text1, text2, tokenizer, max_sequence_length=10),
    batch_size=2,
    shuffle=True,
    collate_fn=dynamic_batching_and_padding_collate_fn
)

In [112]:
for batch in train_dataloader:
    print(batch)

['GLTNAFIASAPAREVRYDGVITPANANYRFMGGDKGGSLTVGSHLTGSNMVTIGPMGVVVFTNNNDYTGNTFIMGGGTLQLGSNTAWGSLPN\n', 'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFPLSPAQLGIWYAQHLDPQVPITIAQYVDLHGALDVEVLERASIDASHELGSGFLRIVERDGEPLQYV\n', 'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFP\n', 'MDRLDFGGGE\n', 'APVPVSGQPVSSE\n'] [4, 3]


TypeError: list indices must be integers or slices, not list

In [ ]:

def dynamic_batching_and_padding_collate_fn(batch):
    pad_token_id = global_tokenizer.pad_token_id()

    # Find the length of the longest sequence in the batch
    max_length = max(len(seq) for seq in batch)

    # Pad each sequence in the batch to max_length
    padded_sequences = [seq + [pad_token_id] * (max_length - len(seq)) for seq in batch]
    data_tokens_padded = torch.tensor(padded_sequences)

    # Create attention masks
    attention_masks = (data_tokens_padded != pad_token_id).long()

    # Create labels by cloning the input tokens and setting the padding tokens to -100
    labels = data_tokens_padded.clone()
    labels[data_tokens_padded == pad_token_id] = -100

    return {
        'input_ids': data_tokens_padded,
        'attention_mask': attention_masks,
        'labels': labels
    }